# Analyse failed trades

Analyse failed trades over multiple trade executors.

 - Show failed and repaired and fix trades
  - Analyse slippage parameters and other causes of failure



In [1]:
monitored_executors = [
    #"https://enzyme-polygon-eth-usdc.tradingstrategy.ai",
    "https://enzyme-polygon-eth-usdc-sls.tradingstrategy.ai",
    "https://enzyme-polygon-matic-usdc.tradingstrategy.ai",
    # "https://enzyme-polygon-multipair.tradingstrategy.ai",
]



In [2]:
import requests
from tradeexecutor.state.state import State

total_downloaded = 0

def download_state(url) -> State:
    global total_downloaded
    resp = requests.get(f"{url}/state")

    if len(resp.content) == 0:
        raise RuntimeError(f"Could not download: {url}")

    try:
        state = State.read_json_blob(resp.text)
    except Exception as e:
        raise RuntimeError(f"Could not decode: {url}") from e
    total_downloaded += len(resp.content)
    return state

states = [download_state(url)for url in monitored_executors]


print(f"Downloaded {total_downloaded:,} bytes")

Downloaded 787,090 bytes


/Users/moo/Library/Caches/pypoetry/virtualenvs/trade-executor-8Oz1GdY1-py3.10/lib/python3.10/site-packages/dataclasses_json/core.py:184: RuntimeWarning: `NoneType` object value of non-optional type initial_cash detected when decoding TradeSummary.
  warnings.warn(
/Users/moo/Library/Caches/pypoetry/virtualenvs/trade-executor-8Oz1GdY1-py3.10/lib/python3.10/site-packages/dataclasses_json/core.py:184: RuntimeWarning: `NoneType` object value of non-optional type strategy_cycle_included_at detected when decoding BalanceEventRef.
  warnings.warn(


In [3]:
from tradeexecutor.state.trade import TradeExecution
from itertools import chain

# Get all trades across different trade executors
all_trades = chain(*[state.portfolio.get_all_trades() for state in states])

t: TradeExecution
failed_trades = [t for t in all_trades if t.is_failed() or t.is_repaired() or t.is_repair_trade()]

print(f"Total {len(failed_trades)} failed and repaired trades")

Total 10 failed and repaired trades


In [4]:
import pandas as pd
from tradeexecutor.analysis.slippage import display_slippage


pd.set_option('display.max_colwidth', None)

df = display_slippage(failed_trades)
display(df)

,Flags,Position,Trade,Lag,Slippage,exactInput amount in,exactInput amount out,Enzyme expected amount,Assumed price,Uniswap price,Failure reason,Tx
5,REP,#3,USDC->WMATIC,11.0,100.0,1.475939,2.269622546029597952,2.247151013201358641,0.650237,0.6503015237410108133085080401,Too little received                                        ,0xbfd1d3b1e1ec6c5ce4bf37aabaf07d6ae8b4eca945e5438db0ce93a6a9534f0b
8,FIX,#3,WMATIC->USDC,,,,,,0.650237,,,
4,REP,#2,WMATIC->USDC,,100.0,4.603424392792025237,2.886265,2.857688,0.627045,0.6269821666929670082936444804,Too little received                                        ,0xc3f674c79fef3fedea13b24e26d811bbb2d49edffd8c1b6c332c81032ef58e52
7,FIX,#2,USDC->WMATIC,,,,,,0.627045,,,
11,REP,#5,USDC->WMATIC,6.0,100.0,6.348186,9.180554910137920512,9.088749270149046272,0.691413,0.6914817309125631585197820098,Too little received                                        ,0xe979c86b847b25757ed25e6a975eb473e7f126a1fadb9f7823f24524622d4e32
12,FIX,#5,WMATIC->USDC,,,,,,0.691413,,,
16,REP,#7,WMATIC->USDC,,100.0,5.849075641784785089,4.90357,4.854255,0.838385,0.8383495615905090608647988530,Too little received                                        ,0xc1a47bdedc8acb78de5eb803af313d3867a3adfa313b55e4d7e07a1fdf3365f0
17,FIX,#7,USDC->WMATIC,,,,,,0.838385,,,
18,REP,#7,WMATIC->USDC,,100.0,5.849075641784785089,4.951599,4.902083,0.846646,0.8465609445408147686826286023,Too little received                                        ,0xcdd25e39fa7277f3eb3402b8a8ba2470bff28b2eb2097cb63abac644261fb4a5
19,FIX,#7,USDC->WMATIC,,,,,,0.846646,,,
